In [1]:
import numpy as np
import pandas as pd
from sklearn import preprocessing

In [2]:
file=pd.read_csv('original.csv',header=None)

## Converting the file into numpy array

In [2]:
file=pd.read_csv('original.csv',header=None)

In [3]:
np_file=np.array(file)

## Preprocessing the Dataset
>  1. Balancing
>  2. Normalizing
>  3. Shiffling


In [4]:
#Finding the number of positive cases

positive=np.sum(np_file[:,11]).astype('int32')
negative=np_file.shape[0]-positive
print('There are {} positive cases and {} negative cases'.format(positive,negative))
#Clearly the dataset is biased with more negative responses

There are 2237 positive cases and 11847 negative cases


In [5]:
count=0
excess=[]
for i in range(np_file.shape[0]):
    if np_file[i,11]==0:
        count+=1
        if count>positive:
            excess.append(i)
balanced=np.delete(np_file,excess,axis=0)

In [6]:
balanced.shape

(4474, 12)

In [7]:
#Finding the number of positive cases

positive=np.sum(balanced[:,11]).astype('int32')
negative=balanced.shape[0]-positive
print('There are {} positive cases and {} negative cases'.format(positive,negative))
#Clearly the dataset is biased with more negative responses

There are 2237 positive cases and 2237 negative cases


## Now the dataset is balanced and now is ready for balancing, shuffling and splitting


## Normalizing

In [8]:
normalized=preprocessing.scale(balanced[:,0:11])

## Applying the shuffling


In [15]:
shuffled_index=np.arange(balanced.shape[0])

In [16]:
np.random.shuffle(shuffled_index)

In [73]:
balanced=balanced[shuffled_index]
normalized=normalized[shuffled_index]

## Splitting the database 

In [74]:
# Count the total number of samples
samples_count = shuffled_index.shape[0]


train_samples_count = int(0.8 * samples_count)
validation_samples_count = int(0.1 * samples_count)

test_samples_count = samples_count - train_samples_count - validation_samples_count



train_input=normalized[0:train_samples_count,]
train_output=balanced[0:train_samples_count,11]

test_input=normalized[train_samples_count:train_samples_count+validation_samples_count,]
test_output=balanced[train_samples_count:train_samples_count+validation_samples_count,11]

validation_input=normalized[train_samples_count+validation_samples_count:,]
validation_output=balanced[train_samples_count+validation_samples_count:,11]


print()


## Saving the data as .npz file


In [75]:
# Save the three datasets in *.npz.
# In the next lesson, you will see that it is extremely valuable to name them in such a coherent way!

np.savez('Audiobooks_data_train', inputs=train_input, targets=train_output)
np.savez('Audiobooks_data_validation', inputs=validation_input, targets=validation_output)
np.savez('Audiobooks_data_test', inputs=test_input, targets=test_output)

## Opening the .npz files

In [76]:
import tensorflow as tf


In [77]:
npz = np.load('Audiobooks_data_train.npz')

# we extract the inputs using the keyword under which we saved them

train_inputs = npz['inputs'].astype(np.float)

# targets must be int because of sparse_categorical_crossentropy (we want to be able to smoothly one-hot encode them)
train_targets = npz['targets'].astype(np.int)

npz = np.load('Audiobooks_data_validation.npz')

validation_inputs, validation_targets = npz['inputs'].astype(np.float), npz['targets'].astype(np.int)

npz = np.load('Audiobooks_data_test.npz')

test_inputs, test_targets = npz['inputs'].astype(np.float), npz['targets'].astype(np.int)

## Creating a model


In [78]:
model=tf.keras.Sequential()

model.add(tf.keras.layers.Dense(50,activation='relu'))
model.add(tf.keras.layers.Dense(50,activation='relu'))
model.add(tf.keras.layers.Dense(2,activation='softmax'))

model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

earlyStop=tf.keras.callbacks.EarlyStopping(patience=2)

model.fit(train_inputs,train_targets,batch_size=100,epochs=100,verbose=1,callbacks=[earlyStop])

Train on 3579 samples
Epoch 1/100
3579/3579 [==============================] - 0s 55us/sample - loss: 0.5559 - accuracy: 0.7019
Epoch 2/100
3579/3579 [==============================] - 0s 12us/sample - loss: 0.4524 - accuracy: 0.7631
Epoch 3/100
3579/3579 [==============================] - 0s 10us/sample - loss: 0.4067 - accuracy: 0.7904
Epoch 4/100
3579/3579 [==============================] - 0s 11us/sample - loss: 0.3807 - accuracy: 0.8050
Epoch 5/100
3579/3579 [==============================] - 0s 10us/sample - loss: 0.3676 - accuracy: 0.7944
Epoch 6/100
3579/3579 [==============================] - 0s 10us/sample - loss: 0.3595 - accuracy: 0.8039
Epoch 7/100
3579/3579 [==============================] - 0s 11us/sample - loss: 0.3515 - accuracy: 0.8039
Epoch 8/100
3579/3579 [==============================] - 0s 10us/sample - loss: 0.3456 - accuracy: 0.8159
Epoch 9/100
3579/3579 [==============================] - 0s 10us/sample - loss: 0.3429 - accuracy: 0.8150
Epoch 10/100
3579/3579 [

3579/3579 [==============================] - 0s 10us/sample - loss: 0.3153 - accuracy: 0.8240
Epoch 27/100
3579/3579 [==============================] - 0s 10us/sample - loss: 0.3129 - accuracy: 0.8360
Epoch 28/100
3579/3579 [==============================] - 0s 9us/sample - loss: 0.3175 - accuracy: 0.8282
Epoch 29/100
3579/3579 [==============================] - 0s 10us/sample - loss: 0.3109 - accuracy: 0.8265
Epoch 30/100
3579/3579 [==============================] - 0s 10us/sample - loss: 0.3136 - accuracy: 0.8296
Epoch 31/100
3579/3579 [==============================] - 0s 10us/sample - loss: 0.3085 - accuracy: 0.8310
Epoch 32/100
3579/3579 [==============================] - 0s 10us/sample - loss: 0.3113 - accuracy: 0.8326
Epoch 33/100
3579/3579 [==============================] - 0s 10us/sample - loss: 0.3085 - accuracy: 0.8379
Epoch 34/100
3579/3579 [==============================] - 0s 9us/sample - loss: 0.3066 - accuracy: 0.8335
Epoch 35/100
3579/3579 [============================

3579/3579 [==============================] - 0s 10us/sample - loss: 0.3014 - accuracy: 0.8340
Epoch 53/100
3579/3579 [==============================] - 0s 10us/sample - loss: 0.2962 - accuracy: 0.8374
Epoch 54/100
3579/3579 [==============================] - 0s 11us/sample - loss: 0.2995 - accuracy: 0.8354
Epoch 55/100
3579/3579 [==============================] - 0s 11us/sample - loss: 0.2969 - accuracy: 0.8391
Epoch 56/100
3579/3579 [==============================] - 0s 11us/sample - loss: 0.2973 - accuracy: 0.8402
Epoch 57/100
3579/3579 [==============================] - 0s 10us/sample - loss: 0.2969 - accuracy: 0.8416
Epoch 58/100
3579/3579 [==============================] - 0s 10us/sample - loss: 0.2955 - accuracy: 0.8427
Epoch 59/100
3579/3579 [==============================] - 0s 9us/sample - loss: 0.2966 - accuracy: 0.8430
Epoch 60/100
3579/3579 [==============================] - 0s 11us/sample - loss: 0.2950 - accuracy: 0.8402
Epoch 61/100
3579/3579 [===========================

3579/3579 [==============================] - 0s 9us/sample - loss: 0.2910 - accuracy: 0.8385
Epoch 78/100
3579/3579 [==============================] - 0s 10us/sample - loss: 0.2922 - accuracy: 0.8410
Epoch 79/100
3579/3579 [==============================] - 0s 10us/sample - loss: 0.2883 - accuracy: 0.8486
Epoch 80/100
3579/3579 [==============================] - 0s 11us/sample - loss: 0.2926 - accuracy: 0.8346
Epoch 81/100
3579/3579 [==============================] - 0s 11us/sample - loss: 0.2920 - accuracy: 0.8365
Epoch 82/100
3579/3579 [==============================] - 0s 11us/sample - loss: 0.2939 - accuracy: 0.8351
Epoch 83/100
3579/3579 [==============================] - 0s 11us/sample - loss: 0.2991 - accuracy: 0.8349
Epoch 84/100
3579/3579 [==============================] - 0s 11us/sample - loss: 0.2911 - accuracy: 0.8382
Epoch 85/100
3579/3579 [==============================] - 0s 11us/sample - loss: 0.2998 - accuracy: 0.8349
Epoch 86/100
3579/3579 [===========================

## Model Testing


In [79]:
model.evaluate(test_inputs,test_targets)

447/447 [==============================] - 0s 144us/sample - loss: 0.3615 - accuracy: 0.7964


[0.3614837938640475, 0.7964206]